In [ ]:
from google.colab import drive
import os
import zipfile
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/Colab_Notebooks/HW5/mydata')
os.getcwd()

!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
AOI_model.h5	 test1.tfrecords  test5.tfrecords     train.tfrecords
logs		 test2.tfrecords  test.csv	      val.tfrecords
result.csv	 test3.tfrecords  test.tfrecords
test0.tfrecords  test4.tfrecords  training.tfrecords


In [ ]:
# !pip install tensorflow_addons

In [ ]:
import numpy as np
import pandas as pd
from time import time

from sklearn.model_selection import train_test_split
import tensorflow as tf
# import tensorflow_addons as tfa
import keras
import keras.layers as layers
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
import datetime
from functools import partial
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.callbacks import Callback
%matplotlib inline
%tensorflow_version 2.x
%load_ext tensorboard
!rm -rf ./logs/

import matplotlib.pyplot as plt


In [ ]:
strategy = tf.distribute.get_strategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

# Find the file name from the folder
# TRAINING_FILENAMES = tf.io.gfile.glob("Train*.tfrecords")
TRAINING_FILENAMES = tf.io.gfile.glob("training.tfrecords")
VAL_FILENAMES = tf.io.gfile.glob("val.tfrecords")
print("Train TFRecord Files:", len(TRAINING_FILENAMES))
print("Validation TFRecord Files:", len(VAL_FILENAMES))

# TEST_FILENAMES = tf.io.gfile.glob("test.tfrecords")
# print("Test TFRecord Files:", len(TEST_FILENAMES))

Number of replicas: 1
Train TFRecord Files: 1
Validation TFRecord Files: 1


In [ ]:
# data loader
def read_tfrecord(example, mode='train'):
  
  # decode the example back to array
  features = (
  {
    "image_raw": tf.io.FixedLenFeature([], tf.string),  # tf.string = bytestring (not text string)
    "Label": tf.io.FixedLenFeature([], tf.int64),   # shape [] means scalar
  }
  if mode != 'test'
  else {"image_raw": tf.io.FixedLenFeature([], tf.string),}
  )
  # decode the TFRecord
  example = tf.io.parse_single_example(example, features)
  image = tf.io.decode_raw(example['image_raw'], tf.uint8)
  # Normalize and reshape data
  image = tf.cast(image, tf.float32) / 255.0 # Normalize  
  image = tf.reshape(image, [512,512,3]) #size of image
  
  if mode == 'train':
    # data augmentation for training set
    image = tf.image.random_crop(image, [380,380,3])
    image = tf.image.random_brightness(image, 0.3)  # 隨機亮度
    image = tf.image.random_saturation(image, 0.7, 1.3)  # 隨機飽和度
    image = tf.image.random_contrast(image, 0.7, 1.3)  # 隨機對比度
    image = tf.image.random_flip_left_right(image)  # 隨機翻左翻右
  
  # resize image for validation and testing set
  else:
    image = tf.image.resize(image, (380, 380)) 
  
  if mode == 'test':
    # only return image feature for testing set
    return image
  else:    
    # get class label and make one hot label
    class_label = tf.cast(example['Label'], tf.int32)
    label = tf.one_hot(class_label, 6) # one hot label
    return image, label 


def get_dataset(filenames, BATCH_SIZE, mode):
  AUTOTUNE = tf.data.experimental.AUTOTUNE
  # Not to keep the order when getting folder
  option_no_order = tf.data.Options()
  option_no_order.experimental_deterministic = False

  # Get the dataset
  dataset = tf.data.TFRecordDataset(filenames)
  dataset = dataset.with_options(option_no_order)
  dataset = dataset.map(
        partial(read_tfrecord, mode = mode), num_parallel_calls=AUTOTUNE
  )

  # dataset = dataset.cache() # This dataset not fits in RAM as well  
  if mode != 'test':
    # testing data set only needed to be read once, so no need repeat/ shuffle
    dataset = dataset.repeat() # repeat until all epoch end
    dataset = dataset.shuffle(16) # 2048
  dataset = dataset.batch(BATCH_SIZE,drop_remainder=False) # make batches, drop_remainder=True
  dataset = dataset.prefetch(AUTOTUNE) # prefetch the first batch

  return dataset

In [ ]:
num_classes = 6
input_shape = (380, 380, 3)
Model = EfficientNetB4(include_top=True, weights=None, input_shape=input_shape, classes=num_classes) # , drop_connect_rate=0.3 (optional)
Model.summary()

Model: "efficientnetb4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 380, 380, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 380, 380, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 380, 380, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 381, 381, 3)  0           normalization[0][0]              
_____________________________________________________________________________________

In [ ]:
'''
# learning rate decay
step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [30, 60], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-3 * schedule(step)
wd = lambda: 1e-4 * schedule(step) # 1e-4

optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)
'''

'\nstep = tf.Variable(0, trainable=False)\nschedule = tf.optimizers.schedules.PiecewiseConstantDecay(\n    [30, 60], [1e-0, 1e-1, 1e-2])\n# lr and wd can be a function or a tensor\nlr = 1e-3 * schedule(step)\nwd = lambda: 1e-4 * schedule(step) # 1e-4\n\noptimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)\n'

In [ ]:
lr = 1e-3
wd = 1e-5
optimizer = keras.optimizers.Adam(lr=lr, decay=wd)

Model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy']) 

In [ ]:
def get_training_dataset(training_filenames, BATCH_SIZE):
  return get_dataset(training_filenames,BATCH_SIZE, mode = 'train')

def get_validation_dataset(validation_filenames, BATCH_SIZE):
  return get_dataset(validation_filenames,BATCH_SIZE, mode = 'val')
EPOCHS = 50
BATCH_SIZE = 8

num_train = 2286
num_val = 242
train_dataset = get_training_dataset(TRAINING_FILENAMES, BATCH_SIZE)
val_dataset = get_validation_dataset(VAL_FILENAMES, BATCH_SIZE)

log_dir = "logs/fit/exp1" 

early_stop_cb = tf.keras.callbacks.EarlyStopping(patience=5, monitor = 'val_accuracy')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
my_callbacks = [tensorboard_callback, early_stop_cb]
Model.fit(train_dataset, steps_per_epoch=int(num_train//BATCH_SIZE), epochs=EPOCHS,
    validation_data=val_dataset, validation_steps=int(num_val//BATCH_SIZE), callbacks=my_callbacks)

Epoch 1/50
285/285 [==============================] - 370s 1s/step - loss: 4.5317 - accuracy: 0.2950 - val_loss: 2.4346 - val_accuracy: 0.2458
Epoch 2/50
285/285 [==============================] - 342s 1s/step - loss: 1.5929 - accuracy: 0.4250 - val_loss: 1.7353 - val_accuracy: 0.3125
Epoch 3/50
285/285 [==============================] - 341s 1s/step - loss: 1.4005 - accuracy: 0.4712 - val_loss: 1.3393 - val_accuracy: 0.4542
Epoch 4/50
285/285 [==============================] - 342s 1s/step - loss: 1.3707 - accuracy: 0.4780 - val_loss: 1.0901 - val_accuracy: 0.5375
Epoch 5/50
285/285 [==============================] - 341s 1s/step - loss: 1.2354 - accuracy: 0.5296 - val_loss: 1.0154 - val_accuracy: 0.6000
Epoch 6/50
285/285 [==============================] - 342s 1s/step - loss: 1.1614 - accuracy: 0.5624 - val_loss: 1.0739 - val_accuracy: 0.5583
Epoch 7/50
285/285 [==============================] - 341s 1s/step - loss: 1.1020 - accuracy: 0.5983 - val_loss: 0.8600 - val_accuracy: 0.6667

In [ ]:
%tensorboard --logdir logs/fit

Validation result

In [ ]:
val_dataset = get_validation_dataset(VAL_FILENAMES, 22)
# image_batch, label_batch = next(iter(test_dataset))
score = Model.evaluate(val_dataset, steps = int(num_val//22))

print('Test loss:', score[0])
print('Test accuracy:', score[1])

11/11 [==============================] - 10s 616ms/step - loss: 0.0508 - accuracy: 0.9752
Test loss: 0.050784625113010406
Test accuracy: 0.9752066135406494


Output predictions for Aidea upload

In [ ]:
TEST_FILENAMES = tf.io.gfile.glob("test.tfrecords")
print("Test TFRecord Files:", len(TEST_FILENAMES))
num_test = 10142
def get_test_dataset(test_filenames, BATCH_SIZE):
  return get_dataset(test_filenames,BATCH_SIZE, mode = 'test')
test_dataset = get_test_dataset(TEST_FILENAMES, 16)
prediction = Model.predict(test_dataset, steps = int(num_test//16)+1)

Test TFRecord Files: 1


In [ ]:
pred = np.argmax(prediction, axis = 1)
print(len(pred))

10142


In [ ]:
df = pd.read_csv('test.csv', header=0)
pred = pd.DataFrame(pred, columns=['Labels'])
df = pd.concat([df, pred], axis=1)
df= df.drop(['Label'], axis=1)
print(df)

                   ID  Labels
0      test_00000.png       1
1      test_00001.png       2
2      test_00002.png       5
3      test_00003.png       0
4      test_00004.png       2
...               ...     ...
10137  test_10137.png       4
10138  test_10138.png       1
10139  test_10139.png       1
10140  test_10140.png       3
10141  test_10141.png       1

[10142 rows x 2 columns]


In [ ]:
df.to_csv('result1.csv')